<a href="https://colab.research.google.com/github/babelPish/slaw/blob/master/spacy_basic/colab_notebooks/05_working_with_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#5. Woking with word vectors
* 바벨피쉬 : Slaw - Spacy 기초
* 김무성

In [1]:
!python -V

Python 3.6.9


#### 참고 
* spaCy official doc. Word Vectors and Semantic Similarity - https://spacy.io/usage/vectors-similarity
* gensim official doc. models.keyedvectors – Store and query word vectors - https://radimrehurek.com/gensim/models/keyedvectors.html
* GENSIM WORD2VEC SIMPLE USAGE - https://gorakgarak.tistory.com/1330
* 딥 러닝을 이용한 자연어 처리 입문 - 영어/한국어 Word2Vec 실습 WikiDocs
3) 영어/한국어 Word2Vec 실습 - https://wikidocs.net/50739
* Pre-trained word vectors of 30+ languages - https://github.com/Kyubyong/wordvectors

-------------

# Installation

In [ ]:
#spacy는 현재 colab에서는 기본 설치되어 있다. 한국어 지원만 다음 과정을 따르면 된다.

In [7]:
# 한국어 형태소 분석을 위한 mecab-ko 설치가 필요하다.
#remove " > /dev/null 2>&1" to see what is going on under the hood
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git > /dev/null 2>&1

In [9]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [11]:
!bash install_mecab-ko_on_colab190912.sh > /dev/null 2>&1

In [12]:
# natto-py를 설치해야 mecab-ko를 spaCy에 연동가능하다.
!pip install natto-py > /dev/null 2>&1

In [13]:
# 한국어 처리 가능여부 테스트. 이 코드가 제대로 실행되면 spaCy 한국어 지원이 제대로 된 것이다.
from spacy.lang.ko import Korean

nlp_ko = Korean()

doc = nlp_ko(u'이것은 문장이다') #Creates a Doc object 

for token in doc:  
    print(token.text, token.pos_, token.tag_) #prints the text and POS

이것 PRON NP
은 ADP JX
문장 NOUN NNG
이 ADP VCP
다 X EC


---------

In [24]:
cd ..

/content


In [26]:
!git clone https://github.com/babelPish/slaw.git

Cloning into 'slaw'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


In [27]:
ls

mecab-0.996-ko-0.9.2/         mecab-ko-dic-2.1.1-20180720.tar.gz  slaw/
mecab-0.996-ko-0.9.2.tar.gz   Mecab-ko-for-Google-Colab/
mecab-ko-dic-2.1.1-20180720/  sample_data/


In [28]:
cd slaw/spacy_basic/colab_notebooks/

/content/slaw/spacy_basic/colab_notebooks


In [29]:
ls

05_working_with_word_vectors.ipynb  models/


In [30]:
cd models

/content/slaw/spacy_basic/colab_notebooks/models


In [31]:
ls

ko_wv/


In [33]:
cd ko_wv

/content/slaw/spacy_basic/colab_notebooks/models/ko_wv


In [34]:
ls

ko.zip


In [35]:
!unzip ko.zip

Archive:  ko.zip
  inflating: ko.bin                  
  inflating: ko.tsv                  


In [36]:
import gensim

In [37]:
model = gensim.models.Word2Vec.load('ko.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [39]:
kv = model.wv

In [40]:
kv.save_word2vec_format('ko_wv.embd', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [41]:
ls

ko.bin  ko.tsv  ko_wv.embd  ko.zip


In [42]:
!python -m spacy init-model ko ko_wv.spacy.embd --vectors-loc ko_wv.embd

✔ Successfully created model
30185it [00:01, 17350.54it/s]
✔ Loaded vectors from ko_wv.embd
✔ Sucessfully compiled vocab
30185 entries, 30185 vectors


In [52]:
ls

ko.bin  ko.tsv  ko_wv.embd  ko_wv.spacy.embd/  ko.zip


In [56]:
ls ko_wv.spacy.embd

meta.json  vocab/


In [59]:
nlp_ko_wv = spacy.load('ko_wv.spacy.embd')

In [60]:
doc1 = nlp_ko_wv('안녕하세요') 

In [63]:
doc2 = nlp_ko_wv('안녕')

In [73]:
doc3 = nlp_ko_wv('반가워')

In [65]:
doc4 = nlp_ko_wv('강아지')

In [66]:
doc1.similarity(doc1)

1.0

In [68]:
doc1.similarity(doc2)

0.3767422436269971

In [74]:
doc1.similarity(doc3)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


0.0

In [77]:
[(w.text, w.pos_) for w in doc3]

[('반가워', 'ADJ')]

In [78]:
doc1.similarity(doc4)

-0.038436487747098264